In [ ]:
import math
from botorch.models.model import Model
from botorch.utils import t_batch_mode_transform
import torch
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models import  SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.acquisition import AnalyticAcquisitionFunction
from botorch.optim.optimize import optimize_acqf
from botorch.optim.initializers import gen_batch_initial_conditions
from botorch.utils.transforms import normalize, unnormalize


problem setting

penicillin function

ouputs:

P 
-t
-CO2

constraints(example):

P >= 12

-t >= -10

-CO2 >= -100

In [ ]:
from botorch.test_functions.multi_objective import Penicillin

test_f = Penicillin(negate= True)

#initialize data
n= 5
random_tensor = torch.zeros((n, 7), dtype= torch.float64)

for i in range(7):
    lower_bound = test_f.bounds[0, i]
    upper_bound = test_f.bounds[1, i]
    random_tensor[:, i] = lower_bound + (upper_bound - lower_bound) * torch.rand(n)

train_X = normalize(random_tensor, test_f.bounds)
train_Y = test_f(random_tensor)
train_Y = train_Y.to(train_X)

acquisition

In [ ]:
class HyperVolumeScalarizedUCB(AnalyticAcquisitionFunction):
    def __init__(
        self,
        model,
        beta: float,
        theta: torch.Tensor,
        ref: torch.Tensor,
        maximize: bool = True,
    ) -> None:
        """
        Initializes the HyperVolume Scalarized Upper Confidence Bound Acquisition Function.

        Args:
            model: A BoTorch model representing the posterior distribution of the objectives.
            beta (Tensor of shape [1] or [o]): The exploration-exploitation trade-off parameter(s).
            theta (Tensor of shape [o]): The weights used for scalarizing the upper bounds, where `o` is the number of objectives.
            maximize (bool): Whether to maximize or minimize the scalarized objective. Defaults to True (maximize).
        """
        super(AnalyticAcquisitionFunction, self).__init__(model)
        self.maximize = maximize
        self.register_buffer("beta", torch.as_tensor(beta))
        self.register_buffer("theta", torch.as_tensor(theta))
        self.register_buffer("ref", torch.as_tensor(ref))
    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Evaluate the scalarized Upper Confidence Bound on the candidate set X.

        Args:
            X (Tensor of shape [b, d]): A tensor containing `(b)` batches of `d`-dimensional design points.

        Returns:
            Tensor of shape [b]: A tensor containing the scalarized Upper Confidence Bound values for each batch.
        """
        self.beta = self.beta.to(X)
        self.theta = self.theta.to(X)
        self.ref = self.ref.to(X)
        posterior = self.model.posterior(X)
        means = posterior.mean.squeeze(dim=-2)  # b x o
        std_devs = posterior.variance.squeeze(dim=-2).sqrt()  # b x o
        m = means.shape[1]
        # Calculate upper confidence bounds for each objective
        u_t = means + (self.beta.expand_as(means) * std_devs) - self.ref # b x o

        # Apply the scalarization function to the upper bounds
        scalarized_ut = torch.min(torch.max(torch.zeros_like(u_t), u_t / self.theta) ** m, dim=-1)[0]  # b

        return scalarized_ut

constraints

In [ ]:
def create_ucb_constraints(model, beta: float, thresholds: torch.Tensor):
    """
    Creates a list of non-linear inequality constraints for a multi-output GP model, ensuring that the upper confidence
    bounds of the model's outputs are greater than or equal to the specified thresholds.

    Args:
        model (MultiTaskGP): A multi-output Gaussian Process model.
        beta (float): The scalar coefficient for the variance component of the UCB.
        thresholds (torch.Tensor): A tensor of thresholds for each output dimension.

    Returns:
        List[Tuple[Callable, bool]]: A list of tuples, each containing a callable constraint and a boolean indicating
                                      whether the constraint is intra-point (True) or inter-point (False). Each callable
                                      takes a tensor `X` of shape [q, d] (where `d` is the dimension of the input space
                                      and `q` can be 1 or more representing different design points) and returns a scalar
                                      that should be non-negative if the constraint is satisfied.
    """

    def make_constraint(i, threshold):
        """
        Creates a constraint function for the i-th objective.

        Args:
            i (int): The index of the output dimension for which to create the constraint.
            threshold (float): The threshold value that the UCB of the i-th output should meet.

        Returns:
            Callable: A function that evaluates the constraint across a batch of design points.
        """
        def constraint(X):
            X = X.unsqueeze(0)
            # Compute posterior at X
            posterior = model.posterior(X)
            mean = posterior.mean[:, i]  # Extract the mean for the i-th output
            variance = posterior.variance[:, i]  # Extract the variance for the i-th output
            ucb = mean + beta * variance.sqrt()  # Compute the UCB

            # Minimum across all points in the batch to satisfy the constraint for any single design point
            return ucb - threshold

        return constraint

    # Create a list of constraints for each output dimension, all set as intra-point since they evaluate individually
    constraints = [(make_constraint(i, thresholds[i]), True) for i in range(thresholds.size(0))]

    return constraints

In [ ]:
def get_random_sample_on_n_sphere(N, R):
    # Return a single sample of a vector of dimension N
    # with a uniform distribution on the (N-1)-Sphere surface of radius R.
    # RATIONALE: https://mathworld.wolfram.com/HyperspherePointPicking.html
    
    # Generate a normally distributed point
    X = torch.randn(N)

    # Normalize this point to the surface of the sphere, then scale by radius R
    return R * X / torch.norm(X)

BO loop

In [ ]:
import warnings
warnings.filterwarnings('ignore')
beta =2
thresholds = torch.tensor([10, -100, -300])
for batch in range(10):
    model_list = []
    m = 3
    for i in range(m):
        current_model = SingleTaskGP(train_X= train_X, train_Y= train_Y[:, i].unsqueeze(-1))
        model_list.append(current_model)
    model = ModelListGP(*model_list)
    #test feasibility maximize UCB function TBD


    #sample theta from distribution
    theta = get_random_sample_on_n_sphere(m,1).abs()
    #create acquisition function
    HVUCB = HyperVolumeScalarizedUCB(model= model, beta= torch.tensor(beta), theta = theta, ref= thresholds)
    #optimize constraint function
    candidate, _ = optimize_acqf(
        acq_function = HVUCB,
        q = 1,
        num_restarts = 5,
        raw_samples = 5,
        nonlinear_inequality_constraints = create_ucb_constraints(beta=beta, model= model, thresholds= thresholds),
        ic_generator = gen_batch_initial_conditions,
        #batch_initial_conditions = torch.tensor([-0.001,1.0], dtype= dtype).view([1,1,-1]),

        #take the standard bounds
        bounds = torch.tensor([[0.0]*7, [1.0]*7]),
        sequential = True
    )
    #update data
    train_X = torch.cat([train_X, candidate],dim=0)
    train_Y = torch.cat([train_Y, test_f(unnormalize(candidate, bounds= test_f.bounds))], dim = 0)

In [ ]:
train_Y